In [607]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
from collections import Counter
import datetime
with open('config/secrets.json') as f:
    config = json.load(f)
db = config['postgres']
con = f"postgresql://{db['user']}:{db['password']}@{db['host']}/{db['dbname']}"
pd.options.mode.copy_on_write = True

## Channel Distribution (sub v.s video)

In [608]:
channel = pd.read_sql('select c.name,c.topic, cl.* from channel_log cl  left join channel c  using(channel_id)',con)
channel = channel.drop_duplicates(subset=['name'],keep='last')
topic_list = channel.topic.to_list()
topic_list = [item.replace(' ','').split(',') for item in topic_list]
topic_list = [item for row in topic_list for item in row]

In [609]:
c = Counter(topic_list)
top_topics = [item[0] for item in c.most_common()[:7]]
print(top_topics)
top_topics[0] = 'Lifestyle'
top_topics[3] = 'News'
print(top_topics)

['Lifestyle_(sociology)', 'Entertainment', 'Food', 'Society', 'Film', 'Politics', 'Music']
['Lifestyle', 'Entertainment', 'Food', 'News', 'Film', 'Politics', 'Music']


In [610]:
def check_category(x):
    for topic in top_topics:
        if topic in x:
            return topic
    return 'Other'
channel['topic'] = channel['topic'].apply(check_category)

In [614]:
fig = px.scatter(channel, x="video_count", y="sub_count",color='topic',
                 log_x=True,
                 size='view_count',
                 labels={
                     "sub_count": "Num of Subscriber",
                     "video_count": "Num of video (log)",
                     "name": "Channel Name",
                     "topic": "Categories",
                     "view_count":"Num of views"
                 },
                 hover_name='name',
                 hover_data={
                    "topic":True,
                    "sub_count":True,
                    "video_count":True,
                    "video_count":False,
                },
                 category_orders={
                              "topic": ["Lifestyle", "Entertainment","Music","Politics","Other"]},
                 title="Top youtube channels in TW",
                #  autosize=False,
                 width=800,
                 height=500,                
                 template="plotly_dark"
                 )
fig.show()

/Users/Harry/Documents/GitHub/top_youtuber_dashboard_tw/.venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [615]:
fig.write_image('images/channel_distribution.png')
fig.write_html('images/channel_distribution.html')

## Channel Distribution (sub v.s video)

In [616]:
stmt = '''
select video_id, published_date, published_time, duration, vl.*
from video_log vl  
join (
select * from video 
where video_type='video') as v
using(video_id)

'''

In [617]:
video = pd.read_sql(stmt,con)

In [618]:
video = video.drop_duplicates(subset='video_id',keep='last')
video['published_date'] = pd.to_datetime(video['published_date'])
video['created_date'] = pd.to_datetime(video['created_date'])
video = video[video['created_date']-video['published_date']>=datetime.timedelta(days=7)]
video['duration'] = pd.to_timedelta(video['duration']).dt.ceil('min')
video = video[video["duration"] < datetime.timedelta(hours=1)]


In [619]:
is_multi = video["duration"].value_counts() > 20
filtered = video[video["duration"].isin(is_multi[is_multi].index)]
filtered['duration'] = filtered.duration.dt.total_seconds()/60
bin_num = 20
filtered['time_binning'] = pd.cut(filtered['duration'],bins=bin_num,
                                  include_lowest=True,
                                  precision=0
                                #   labels=list(range(bin_num))
                                  )

In [620]:
bin_names = [f"{int(temp.left)}~{int(temp.right)}" for temp in filtered['time_binning'].dtype.categories]

In [709]:
trace1 = go.Bar(x = bin_names,
                     y=filtered.groupby('time_binning',observed=True)['duration'].count(),
                     name='Number of video',
                     yaxis='y1',
                     marker=dict(color='#1D1B86')
                     )
trace2 = go.Scatter(x=bin_names,
                      y=filtered.groupby('time_binning',observed=True)['view_count'].median().sort_index().values,
                      name='Video views median',
                      yaxis='y2',
                      mode='lines+markers',
                      marker_size=8
                      )
data = [trace1, trace2]
layout = go.Layout(title='Video performance base on duration (min)',
                    yaxis=dict(title='num'),
                    yaxis2=dict(title='views',
                                overlaying='y',
                                side='right'),
                    template="plotly_dark")
fig = go.Figure(data=data,layout=layout,)

fig.update_layout(legend=dict(
    y=1.15,
    x=0.75),
    autosize=False,
    width=800,
    height=500
    )

fig.show()

In [710]:
fig.write_image('images/duration_view_relation.png')
fig.write_html('images/duration_view_relation.html')

## View-Sub ratio

In [624]:
stmt = '''
with channel_latest as (
select *
from channel_log cl 
where created_date = (
select max(created_date) from channel_log))

select c.name,cl.sub_count,v.video_type,v.published_date, vl.*
from video_log vl  
join video v
using(video_id)
join channel c
using(channel_id)
join channel_latest cl
using(channel_id)
'''

In [625]:
video = pd.read_sql(stmt,con)
video = video.drop_duplicates(subset='video_id',keep='last')

In [626]:
video['published_date'] = pd.to_datetime(video['published_date'])
video['created_date'] = pd.to_datetime(video['created_date'])
video = video[video['created_date']-video['published_date']>datetime.timedelta(days=30)]
video['view_sub_ratio'] = video['view_count']/video['sub_count']

In [627]:
exclude_list = ['周杰倫 Jay Chou','凱文羊','福茂唱片']
video = video[~video['name'].isin(exclude_list)]

In [628]:
video_ratio_top_10 = video[video['video_type']=='video'
                           ].groupby('name')['view_sub_ratio'].mean().sort_values(ascending=False)[:10]
video_ratio_top_10.index

Index(['HOOK', '好棒Bump', 'Ku's dream酷的夢-', '人生肥宅x尊', 'cheap', '欸你這週要幹嘛',
       '這群人TGOP', '千千進食中', '胡子Huzi', '啾啾鞋'],
      dtype='object', name='name')

In [699]:
trace1 = go.Scatter(x=video_ratio_top_10.index.tolist(),
                      y=video[video['name'].isin(video_ratio_top_10.index)].drop_duplicates(subset='name').set_index('name').reindex(video_ratio_top_10.index).sub_count.values,
                      name='Subscriber Count',
                      yaxis='y1',
                      mode='markers',
                      marker_color="#ffe476",
                      marker_size=8
                      )
# trace2 = go.Bar(x = video_ratio_top_10.index.tolist(),
#                      y=avg_video_view.reindex(video_ratio_top_10.index.tolist()),
#                      name='Average views',
#                      yaxis='y2',
#                      marker_color="#64656C",
#                      )
boxs = []
for name in video_ratio_top_10.index.tolist():
    boxs.append(go.Box(
                y=video[(video['video_type']=='video') & (video['name']==name)]['view_sub_ratio'],
                boxpoints=False,
                yaxis='y2',
                name=name,
                marker_color="#7B4173",
                showlegend=False,
                ))

dummy = go.Box(
    x=[None],
    y=[None],
    name="Video Views",
    marker_color="#7B4173"
)

data = [trace1,*boxs,dummy]
layout = go.Layout(title='Video Performance (past 30 days)',
                    yaxis=dict(title='Subscribers',
                               overlaying='y2',
                                side='right'),
                    yaxis2=dict(title='Video views',
                                ),
                    template="plotly_dark")
                    
fig = go.Figure(data=data,layout=layout,)

fig.update_layout(
    legend=dict(
    y=1.15,
    x=1.05),
    autosize=False,
    width=800,
    height=500
    )

fig.show()

In [700]:
fig.write_image('images/view_sub_ratio.png')
fig.write_html('images/view_sub_ratio.html')